In [1]:
from uti.datablock import *

In [2]:
x_train,y_train,x_valid,y_valid = get_data()

/home/jupyter/.fastai/data/mnist.pkl.gz


In [4]:
train_ds,valid_ds = get_dataset(x_train,y_train,x_valid,y_valid)

In [5]:
data = Databunch(*get_dl(train_ds,valid_ds,bs=64))

In [22]:
data.c = len(y_train.unique())
data.c

10

In [14]:
#export
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

In [10]:
#export
def get_model(data,lr=0.5,nh=50):
    m = data.train_ds.tensors[0].shape[1]
    model = nn.Sequential(nn.Linear(m,nh),nn.ReLU(),nn.Linear(nh,data.c))
    opt = optim.SGD(model.parameters(),lr=lr)
    return model,opt

class Learner():
    def __init__(self, model,opt,loss_func,data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

In [12]:
loss_func = F.cross_entropy

In [23]:
model,opt = get_model(data)

In [24]:
learn = Learner(model,opt,loss_func,data)

In [25]:
def fit(epochs,learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            batch_preds = learn.model(xb)
            loss = learn.loss_func(batch_preds,yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()
            
        learn.model.eval()
        with torch.no_grad():
            total_loss, accuracy = 0.0, 0.0
            for xb,yb in learn.data.valid_dl:
                valid_preds = learn.model(xb)
                total_loss += learn.loss_func(valid_preds,yb)
                accuracy += (torch.argmax(valid_preds,dim=1) == yb).float().mean()
        
        nv = len(learn.data.valid_dl)
        print(epoch, total_loss/nv, accuracy/nv)
    return total_loss/nv, accuracy/nv

In [27]:
loss,acc = fit(4,learn)

0 tensor(0.1114) tensor(0.9692)
1 tensor(0.0998) tensor(0.9728)
2 tensor(0.1072) tensor(0.9694)
3 tensor(0.1037) tensor(0.9717)
